In [ ]:
import tensorflow as tf

# import tensorflow_io as tfio

import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import seaborn as sns
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from IPython import display

print(tf. __version__)
import seaborn as sns
import librosa.display

import soundfile
import statistics
import librosa as lb
# print(tfio. __version__)
import shutil
import random
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.preprocessing import LabelEncoder

from datetime import datetime


: 

In [ ]:
dataSetPath = '../processed_audio_files_nn/'
#dataSetPath = 'processed_audio_files/'
disease_labels = ["Asthma","COPD","Bronchiectasis","Pneumonia","URTI","Healthy","Bronchiolitis","LRTI"]
target_conditions = ["COPD","Healthy"]

: 

In [ ]:
def getFilenameInfo(file):
    return file.split('_')

: 

Following loop will run through the dataset path and and analyse it


In [ ]:
def plotDataSet(dataSetPath , disease_labels):
  '''plots the data-set's ingredients, and returns a dictionary with the results'''
  audio_data_dir = dataSetPath + "*.wav"
  filenames = os.listdir(dataSetPath)
  dataCountList = [0 for i in range(len(disease_labels))]
  for filename in filenames:
    dataCountList[disease_labels.index(getFilenameInfo(filename)[0])] += 1

  # print(dataCountList,dataCountList)

  plt.bar(disease_labels,dataCountList)
  plt.xticks(rotation=90)
  dataSetInfo = {disease_labels[i] : dataCountList[i] for i in  range(len(dataCountList))}    #dictionary with data set consistancy
  return dataSetInfo

: 

## Data set Visualization

In [ ]:
plotDataSet(dataSetPath , disease_labels)

: 

## Data pre-process

copy the required files to a specific location


In [ ]:
def copy_targetConditions(targets = [], destination = 'target_conditions/',source = 'balanced_dataset/'):
    '''copy the files with names in target conditions list to the destination location'''
    files = os.listdir(source)
    if not os.path.exists(destination):
        os.makedirs(destination)
    for file in files:
        if (getFilenameInfo(file)[0] in targets):
            # print(getFilenameInfo(file)[0])
            shutil.copy(source + file , destination)
            


: 

In [ ]:
 #copy_targetConditions(targets = target_conditions, destination = 'target_conditions/',source = 'balanced_dataset/')

: 

In [ ]:
plotDataSet('../target_conditions/' , target_conditions)

: 

In [ ]:
max_pad_len = 862 # to make the length of all MFCC equal

def extract_features(file_name):
    """
    This function takes in the path for an audio file as a string, loads it, and returns the MFCC
    of the audio"""
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast', duration=20) 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccs

: 

load data

In [ ]:
dataSetPath = '../target_conditions/'

files = os.listdir(dataSetPath)
# shuffle the data
random.shuffle(files)

labels = np.array([getFilenameInfo(file)[0] for file in files])
# print (str(labels[:10]) + 'files : ' + str(files[:10]))

# print(files)

: 

## Feature Extraction

In [ ]:
features = []

for file in files:
    features.append(extract_features(dataSetPath+file))

features = np.array(features,dtype=float )
print('feature extraction finished with : ' , len(features) , "features.")



: 

In [ ]:
plt.figure(figsize=(10, 4))
librosa.display.specshow(features[0], x_axis='time')
plt.colorbar()
plt.title('MFCC')
plt.tight_layout()
plt.show()

: 

## Encode and Split

In [ ]:
le = LabelEncoder()
i_labels = le.fit_transform(labels)
oneHotLabels = to_categorical(i_labels)
# print(oneHotLabels[:10])

features = np.reshape(features,(*features.shape,1))


#split the data set

x_train,x_test,y_train,y_test = train_test_split(features,oneHotLabels,stratify=oneHotLabels,test_size=0.3,random_state = 42)
x_test,x_val,y_test,y_val = train_test_split(features,oneHotLabels,stratify=oneHotLabels,test_size=0.2,random_state = 42)


tensor_x_test = tf.convert_to_tensor(x_test)
print(len(x_val))


: 

## Model creation

In [ ]:
num_rows = 40
num_columns = 862
num_channels = 1

num_labels = oneHotLabels.shape[1]
filter_size = 2

# Construct model 
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=filter_size,
                 input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=filter_size, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=filter_size, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=filter_size, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax')) 

: 

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 

: 

In [ ]:
# Display model architecture summary 
model.summary()


# Calculate pre-training accuracy 
print(type(x_test))
tensor_y_test = tf.convert_to_tensor(y_test)


score = model.evaluate(tensor_x_test, tensor_y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

: 

In [ ]:
array = np.array([[1,2,3],[3,4,5],[5,6,7]])
print("This is a numpy array:")
print(array, array.shape, type(array))
tensor = tf.convert_to_tensor(array)
print("Converted numpy array into tensor:")
print(tensor)

: 

## Model Training

In [ ]:
# train model
num_epochs = 100
num_batch_size = 128

callbacks = [
    ModelCheckpoint(
        filepath='mymodel2_{epoch:02d}.h5',
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_accuracy` score has improved.
        save_best_only=True,
        monitor='val_accuracy',
        verbose=1)
]
start = datetime.now()

history = model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs,
          validation_data=(x_test, y_test), callbacks=callbacks, verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

: 

In [ ]:
metrics = history.history
plt.plot(history.epoch, metrics['loss'], metrics['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

: 

## Testing

In [ ]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

: 

In [ ]:
preds = model.predict(x_test) # label scores 

classpreds = np.argmax(preds, axis=1) # predicted classes 

y_testclass = np.argmax(y_test, axis=1) # true classes

n_classes=2 # number of classes

: 

In [ ]:


print(confusion_matrix(y_testclass, classpreds))

confusion_mtx = tf.math.confusion_matrix(y_testclass, classpreds)
plt.rcParams.update({'font.size': 22})
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx,
            xticklabels=target_conditions,
            yticklabels=target_conditions,
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()

: 

## Run Inferance

In [ ]:
sample_file = '../target_conditions/Healthy_187_1b1_Ll_sc_Meditron_copy_21.wav'
sample_ds = []
sample_ds.append(extract_features(sample_file))
sample_ds = np.array(sample_ds)
sample_ds = np.reshape(sample_ds,(*sample_ds.shape,1))
# print(sample_ds)
prediction = model.predict(sample_ds)
print(prediction)
plt.bar(target_conditions, tf.nn.softmax(prediction[0]))
  # plt.title(f'Predictions for "{disease_labels[label[0]]}"')
  # plt.show()

: 